In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
from pathlib import Path
import sys
sys.path.append('../src')
from utils.get_prices import *


DATA_PATH = Path('../data')
DATA_PATH_BASKETS = DATA_PATH / 'baskets'
DATA_PATH_RAW = DATA_PATH / 'raw'
DATA_PATH_PROCESSED = DATA_PATH / 'processed'

In [2]:
basket_name = 'all_tickers_20201224'
basket_path = DATA_PATH_BASKETS / (basket_name + '.csv')
tickers = pd.read_csv(basket_path, header=None, names=['Ticker'], squeeze=True)


In [3]:
tickers

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
       ... 
722    ZION
723      ZM
724      ZS
725     ZTS
726     ZUO
Name: Ticker, Length: 727, dtype: object

In [4]:
%%time

start = '2020-01-01' # start must be at least [window]Ds earlier than bt_start
end = '2020-12-24'

timestamp = start.replace('-', '') + '_' + end.replace('-', '')

price_filename = '_'.join(['prc', basket_name, timestamp]) + '.csv'
price_filepath = DATA_PATH_RAW / price_filename

if Path(price_filepath).exists():
    print("Found existing price file. Reading...")
    prices = pd.read_csv(price_filepath, header=[0, 1], index_col=0)
    print("Prices read from: ", price_filepath)
else:
    prices = get_prices(tickers, start, end,
                        types=['Adj Close', 'High'],
                        out_path=price_filepath,
                        sort_tks=True)

Gaps found and filled in  ('Adj Close', 'ARNC')  :
['20201001', '20201002']
Gaps found and filled in  ('Adj Close', 'LUMN')  :
['20200625', '20200626', '20200629', '20200630', '20200701', '20200702', '20200706', '20200707', '20200708', '20200709', '20200710', '20200713', '20200714', '20200715', '20200716', '20200717', '20200720', '20200721', '20200722', '20200723', '20200724', '20200727', '20200728', '20200729', '20200730', '20200731', '20200803', '20200804', '20200805', '20200806', '20200807', '20200810', '20200811', '20200812', '20200813', '20200814', '20200817', '20200818', '20200819', '20200820', '20200821', '20200824', '20200825', '20200826', '20200827', '20200828', '20200831', '20200901', '20200902', '20200903', '20200904', '20200908', '20200909', '20200910', '20200911', '20200914', '20200915', '20200916', '20200917']
Gaps found and filled in  ('High', 'ARNC')  :
['20201001', '20201002']
Gaps found and filled in  ('High', 'LUMN')  :
['20200625', '20200626', '20200629', '20200630'

In [5]:
prices.tail()

Attributes   Adj Close                                                        \
Symbols              A    AAL         AAP        AAPL        ABBV        ABC   
Date                                                                           
2020-12-18  119.300003  16.51  159.990005  126.660004  104.449997  98.610001   
2020-12-21  117.779999  16.10  160.490005  128.229996  103.709999  97.669998   
2020-12-22  117.370003  15.48  159.399994  131.880005  102.800003  96.720001   
2020-12-23  117.300003  15.89  159.289993  130.960007  103.279999  97.239998   
2020-12-24  117.309998  15.66  160.679993  131.970001  103.260002  96.970001   

Attributes                                                 ...        High  \
Symbols           ABMD         ABT         ACN       ACWI  ...           Z   
Date                                                       ...               
2020-12-18  293.390015  108.970001  266.250000  89.779999  ...  140.647995   
2020-12-21  295.809998  108.089996  261.420013  89.209999  ...  136.800003   
2020-12-22  302.329987  108.279999  259.470001  89.059998  ...  144.300003   
2020-12-23  304.049988  107.449997  257.440002  89.480003  ...  142.350006   
2020-12-24  303.410004  108.349998  257.299988  89.589996  ...  143.083496   

Attributes                                                             \
Symbols            ZBH        ZBRA         ZEN          ZG       ZION   
Date                                                                    
2020-12-18  152.419998  378.500000  142.279999  144.130005  41.860001   
2020-12-21  150.240005  376.730011  141.080002  141.550003  42.560001   
2020-12-22  151.699997  387.279999  144.188004  149.440002  42.860001   
2020-12-23  152.300003  384.600006  144.500000  147.990005  43.700001   
2020-12-24  149.699997  382.279999  144.559998  148.130005  43.619999   

Attributes                                              
Symbols             ZM          ZS         ZTS     ZUO  
Date                                                    
2020-12-18  408.109985  198.160004  164.539993  12.900  
2020-12-21  427.759003  200.399994  161.559998  13.240  
2020-12-22  413.500000  208.065002  162.380005  14.080  
2020-12-23  402.600006  212.585007  162.979996  14.135  
2020-12-24  384.880005  209.979996  161.869904  14.600  

[5 rows x 1454 columns]

In [6]:
prices_ad = prices['Adj Close']
prices_hi = prices['High']

In [7]:
window_size = 22
thld_rtn = 0.1
backtest_start = '2020-07-01'
timestamp_backtest = backtest_start.replace('-', '') + '_' + end.replace('-', '')

In [13]:
roll_rtns = prices_ad.pct_change(window_size)
roll_rtns_shifted = roll_rtns.shift(-window_size).loc[backtest_start:].dropna(how='all')

In [25]:
n_trades = roll_rtns_shifted.count(axis=0) # count non NA
n_trades.name = 'N_Trades'
n_hits = (roll_rtns_shifted >= thld_rtn).sum(axis=0, skipna=True)
hit_rate = n_hits / n_trades
hit_rate.name = ''.join(['% Probability of Price = Close+',
                         format(thld_rtn*100, '.0f'),
                         '%, Close+22D From Trade'])
res = pd.concat([hit_rate, n_trades], axis=1)

# save results to file
filename = '_'.join(['prob_rtn_over', format(thld_rtn, '.2f'),
                     'over', str(window_size)+'d', timestamp_backtest])
filepath = DATA_PATH_PROCESSED / (filename + '.csv')
res.to_csv(filepath)

In [26]:
res

,"% Probability of Price = Close+10%, Close+22D From Trade",N_Trades
Symbols,,
A,0.049020,102
AAL,0.352941,102
AAP,0.117647,102
AAPL,0.303922,102
ABBV,0.235294,102
...,...,...
ZION,0.313725,102
ZM,0.431373,102
ZS,0.362745,102
